In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pycaret --ignore-installed llvmlite

In [ ]:
from pycaret.regression import *


In [ ]:
import os, cv2, re, random
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
def broad_analysis(data):
    print('shape of the dataset')
    print(data.shape)
    print('============================================================')
    print('============================================================')
    print('columns in the dataset')
    print(data.columns)
    print('============================================================')
    print('============================================================')
    print('infos on the dataset')
    i = 0
    features_list = df.columns
    while i < len(df.columns):
        print(features_list[i])
        print(df[str(features_list[i])].unique())
        i += 1
    print('============================================================')
    print('============================================================')
    print('infos on the type repartition')
    print(df.dtypes.value_counts())
    print('============================================================')
    print('============================================================')
    print(data.info())
    print('============================================================')
    print('============================================================')
    print('head')
    print(data.head())
    print('============================================================')
    print('============================================================')
    print('tail')
    print(data.tail())
    print('============================================================')
    print('============================================================')    
    print('null data')
    print(data.isnull().any())
    print('============================================================')
    print('============================================================')
    print('description')
    print(np.round(data.describe()))
    print('============================================================')
    print('============================================================')
    print('counting_values')    
    
def remove_unique_feature(df):
    i = 0
    features_list = df.columns
    while i < len(features_list):
        if len(df[features_list[i]].unique()) == 1:
            print('dropping: ',features_list[i])
            df.drop(features_list[i], 1, inplace=True)
        i += 1
    return df

from PIL import Image
  
def most_common_used_color(img):
    # Get width and height of Image
    width, height = img.size
    # Initialize Variable
    r_total = 0
    g_total = 0
    b_total = 0
    count = 0
    # Iterate throught each pixel
    for x in range(0, width):
        for y in range(0, height):
            # r,g,b value of pixel
            r, g, b = img.getpixel((x, y))
            r_total += r
            g_total += g
            b_total += b
            count += 1
    return (r_total/count, g_total/count, b_total/count)

In [ ]:
from tensorflow import keras
model = keras.models.load_model('/kaggle/input/cats-vs-dogs/model_keras.h5')

In [ ]:
df = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")
broad_analysis(df)
df = df.drop_duplicates()
df = remove_unique_feature(df)
print(df.shape)

In [ ]:
df_test = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv")
broad_analysis(df_test)
print(df_test.shape)

In [ ]:
files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if os.path.join(dirname, filename) not in ["/kaggle/input/petfinder-pawpularity-score/sample_submission.csv","/kaggle/input/petfinder-pawpularity-score/train.csv","/kaggle/input/petfinder-pawpularity-score/test.csv","/kaggle/input/cats-vs-dogs/model_keras.h5"] and dirname != "/kaggle/input/petfinder-pawpularity-score/test":
            files.append(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
batch_size = 16
img_width = 150 
img_height = 150 

x = []
red = []
green = []
blue = []
for imgs in files:
    img = mpimg.imread(imgs)
    img = cv2.resize(img, (img_width,img_height), interpolation=cv2.INTER_CUBIC)
    x.append(img)
    img = Image.open(imgs)
    # Convert Image into RGB
    img = img.convert('RGB')
    # call function
    common_color = most_common_used_color(img)
    red.append(common_color[0])
    green.append(common_color[1])
    blue.append(common_color[2])

val_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = val_datagen.flow(np.array(x), batch_size=batch_size)
prediction_probabilities = model.predict_generator(test_generator, verbose=1)

In [ ]:
animal = []
for p in prediction_probabilities:
    if p[0] < 0.5:
        animal.append(0)
    else:
        animal.append(1)

In [ ]:
df['animal'] = animal
df['red'] = red
df['green'] = green
df['blue'] = blue

In [ ]:
files = df_test.Id.values.tolist()
print(files)
i = 0
while i < len(files):
    files[i] = '/kaggle/input/petfinder-pawpularity-score/test/'+files[i]+'.jpg'
    i += 1
print(files)
    

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
batch_size = 16
img_width = 150 
img_height = 150 

x = []
red = []
green = []
blue = []
for imgs in files:
    img = mpimg.imread(imgs)
    img = cv2.resize(img, (img_width,img_height), interpolation=cv2.INTER_CUBIC)
    x.append(img)
    img = Image.open(imgs)
    # Convert Image into RGB
    img = img.convert('RGB')
    # call function
    common_color = most_common_used_color(img)
    red.append(common_color[0])
    green.append(common_color[1])
    blue.append(common_color[2])

val_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = val_datagen.flow(np.array(x), batch_size=batch_size)
prediction_probabilities = model.predict_generator(test_generator, verbose=1)

In [ ]:
animal = []
for p in prediction_probabilities:
    if p[0] < 0.5:
        animal.append(0)
    else:
        animal.append(1)

In [ ]:
df_test['animal'] = animal
df_test['red'] = red
df_test['green'] = green
df_test['blue'] = blue

In [ ]:
import pandas as pd
import numpy as np
import math
from plotnine import *
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import preprocessing


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import decomposition
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import re
from sklearn.preprocessing import StandardScaler
import math


In [ ]:

def visualise_correlation(data):
    correlation = data.corr()
    plt.figure(figsize=(14, 12))
    heatmap = sns.heatmap(correlation, annot=True, linewidths=0, vmin=-1, cmap="RdBu_r")
    
def PCA_generator(df):
    i = 0
    j = 0
    size = len(df.columns)
    X = df.values
    X_std = StandardScaler().fit_transform(X)
    mean_vec = np.mean(X_std, axis=0)
    cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
    print('Covariance matrix \n%s' %cov_mat)
    cov_mat = np.cov(X_std.T)
    eig_vals, eig_vecs = np.linalg.eig(cov_mat)
    print('Eigenvectors \n%s' %eig_vecs)
    print('\nEigenvalues \n%s' %eig_vals)
    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
    eig_pairs.sort(key=lambda x: x[0], reverse=True)
    print('Eigenvalues in descending order:')
    for i in eig_pairs:
        print(i[0])
    tot = sum(eig_vals)
    var_exp_sorted = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
    cum_var_exp_sorted = np.cumsum(var_exp_sorted)
    with plt.style.context('seaborn-whitegrid'):
        plt.figure(figsize=(size, 4))
        plt.bar(range(size), var_exp_sorted, alpha=0.5, align='center', label='individual explained variance')
        plt.step(range(size), cum_var_exp_sorted, where='mid', label='cumulative explained variance')
        plt.ylabel('Explained variance ratio')
        plt.xlabel('Principal components')
        plt.legend(loc='best')
        plt.tight_layout()
        plt.show()
    pca = PCA(n_components=size)
    pca.fit(X_std)
    print('variance ratio ',pca.explained_variance_ratio_) 

def pca_components(df, nb_components):
    X = df.values
    std_scale = preprocessing.StandardScaler().fit(X)
    X_scaled = std_scale.transform(X)
    pca = decomposition.PCA(n_components=nb_components)
    pca.fit(X_scaled) 
    print (pca.explained_variance_ratio_)
    print (pca.explained_variance_ratio_.sum())
    X_projected = pca.transform(X_scaled)  
    pcs = pca.components_

 
def PCA_representation(df):
    X = df.values
    std_scale = preprocessing.StandardScaler().fit(X)
    X_scaled = std_scale.transform(X)
    pca = decomposition.PCA(n_components=2)
    pca.fit(X_scaled)
    print(pca.explained_variance_ratio_)
    print(pca.explained_variance_ratio_.sum())
    X_projected = pca.transform(X_scaled)
    pcs = pca.components_
    for i, (x, y) in enumerate(zip(pcs[0, :], pcs[1, :])):
        plt.plot([0, x], [0, y], color='k')
        plt.text(x, y, df.columns[i], fontsize='14')
    plt.plot([-0.7, 0.7], [0, 0], color='grey', ls='--')
    plt.plot([0, 0], [-0.7, 0.7], color='grey', ls='--')
    plt.xlim([-0.7, 0.7])
    plt.ylim([-0.7, 0.7])

In [ ]:
df = df[[ 'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur', 'Pawpularity',
       'animal', 'red', 'green', 'blue']]
label = df.Pawpularity.values.tolist()
print(list(set(df.Action.values.tolist())))

In [ ]:
df_test = df_test[[ 'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur',
       'animal', 'red', 'green', 'blue']]
print(list(set(df_test.Action.values.tolist())))

In [ ]:
df2 = df[['Subject Focus', 'Face', 'Near', 'Accessory','Action',
       'Group', 'Collage', 'Occlusion', 'Info', 'Blur',
       'animal', 'red', 'green', 'blue']]

df2_test = df_test[['Subject Focus', 'Face', 'Near', 'Accessory','Action',
       'Group', 'Collage', 'Occlusion', 'Info', 'Blur',
       'animal', 'red', 'green', 'blue']]

In [ ]:
reg = setup(data = df, 
             target = 'Pawpularity',
             normalize = True,
             silent = True)

In [ ]:
compare_models()

In [ ]:
model  = create_model('omp')      

In [ ]:
tuned_model = tune_model(model)

interpret_model(tuned_model)

In [ ]:
predictions = predict_model(tuned_model, data = df_test)
df_test['Pawpularity'] = predictions['Label']
df_test.to_csv('submission.csv',index=False)
df_test.head()